# Broadcasting
- [Video](https://www.youtube.com/watch?v=PGC0UxakTvM&feature=youtu.be)
- [Matrix Multiplication](matrixmultiplication.xyz) provides a nice visualization of matrix multiplcation

## Aside on torch and cuda

In [35]:
import numpy as np
import torch
#from fastai.core import T
from torch.cuda import Tensor as T, LongTensor as LT
import torch.nn

In [46]:
torch.__version__

'0.4.0'

In [10]:
t = torch.randn(4,5) ; t


 1.9128  0.1609 -1.9932 -1.6115 -0.2036
-0.8628  1.6235 -0.4426 -0.0067  0.5113
 1.0048 -0.9494 -0.4952  0.5479 -0.7674
-0.5348  0.5283 -0.6200 -0.5942  1.3887
[torch.FloatTensor of size 4x5]

In [11]:
t.shape

torch.Size([4, 5])

In [12]:
t.cuda()*3


 5.7383  0.4827 -5.9797 -4.8346 -0.6107
-2.5885  4.8705 -1.3277 -0.0202  1.5340
 3.0143 -2.8482 -1.4857  1.6438 -2.3023
-1.6045  1.5849 -1.8599 -1.7826  4.1662
[torch.cuda.FloatTensor of size 4x5 (GPU 0)]

## Aside about Broadcasting and Matrix Multiplication

Now let's dig in to what we were doing with `torch.matmul`: matrix multiplication.  First, let's start with a simpler building block: **broadcasting**.

### Element-wise operations 

Broadcasting and element-wise operations are supported in the same way by both numpy and pytorch.

Operators (+,-,\*,/,>,<,==) are usually element-wise.

Examples of element-wise operations:

In [37]:
a = np.array([10, 6, -4])
b = np.array([2, 8, 7])
a,b

(array([10,  6, -4]), array([2, 8, 7]))

In [45]:
a > 0

array([ True,  True, False])

In [21]:
%time a + b

CPU times: user 25 µs, sys: 9 µs, total: 34 µs
Wall time: 43.6 µs


array([12, 14,  3])

In [22]:
(a < b).mean()

0.6666666666666666

In [23]:
type(a), type(b)

(numpy.ndarray, numpy.ndarray)

In [38]:
at = LT(a)
bt = LT(b)
at, bt

(tensor([ 10,   6,  -4], device='cuda:0'),
 tensor([ 2,  8,  7], device='cuda:0'))

a = T([10, 6, -4]).cuda()
b = T([2, 8, 7]).cuda()
a,b

In [39]:
type(at)

torch.Tensor

In [34]:
%time at+bt

CPU times: user 240 µs, sys: 90 µs, total: 330 µs
Wall time: 191 µs


tensor([ 12,  14,   3], device='cuda:0')

### Broadcasting

The term **broadcasting** describes how arrays with different shapes are treated during arithmetic operations.  The term broadcasting was first used by Numpy, although is now used in other libraries such as [Tensorflow](https://www.tensorflow.org/performance/xla/broadcasting) and Matlab; the rules can vary by library.

From the [Numpy Documentation](https://docs.scipy.org/doc/numpy-1.10.0/user/basics.broadcasting.html):
*broadcasting* describes how numpy treats arrays with different shapes during arithmetic operations. 
Subject to certain constraints, the smaller array is “broadcast” across the larger 
array so that they have compatible shapes. <br>
Broadcasting provides a means of vectorizing array operations so that looping occurs in C instead of Python. 
It does this without making needless copies of data and usually leads to efficient algorithm implementations.
<br>
In addition to the efficiency of broadcasting, it allows developers to write less code, 
which typically leads to fewer errors.
<br>
*This section was adapted from [Chapter 4](http://nbviewer.jupyter.org/github/fastai/numerical-linear-algebra/blob/master/nbs/4.%20Compressed%20Sensing%20of%20CT%20Scans%20with%20Robust%20Regression.ipynb#4.-Compressed-Sensing-of-CT-Scans-with-Robust-Regression) of the fast.ai [Computational Linear Algebra](https://github.com/fastai/numerical-linear-algebra) course.*

#### Broadcasting with a scalar

In [41]:
at

tensor([ 10,   6,  -4], device='cuda:0')

In [42]:
at > 0

tensor([ 1,  1,  0], dtype=torch.uint8, device='cuda:0')

How are we able to do a > 0?  0 is being **broadcast** to have the same dimensions as a.

Remember above when we normalized our dataset by subtracting the mean (a scalar) from the entire data set (a matrix) and dividing by the standard deviation (another scalar)?  We were using broadcasting!

Other examples of broadcasting with a scalar:

In [43]:
a + 1

array([11,  7, -3])

In [48]:
m = np.array([[1, 2, 3], [4,5,6], [7,8,9]]); m

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [49]:
2*m

array([[ 2,  4,  6],
       [ 8, 10, 12],
       [14, 16, 18]])

#### Broadcasting a vector to a matrix

We can also broadcast a vector to a matrix:

In [50]:
c = np.array([10,20,30]); c

array([10, 20, 30])

In [51]:
m + c

array([[11, 22, 33],
       [14, 25, 36],
       [17, 28, 39]])

In [25]:
c + m

array([[11, 22, 33],
       [14, 25, 36],
       [17, 28, 39]])

Although numpy does this automatically, you can also use the `broadcast_to` method:

In [26]:
c.shape

(3,)

`broadcast_to` takes a vector and broadcasts it to that shape,
and shows us what that would look like.

In [27]:
np.broadcast_to(c[:,None], m.shape)

array([[10, 10, 10],
       [20, 20, 20],
       [30, 30, 30]])

In [28]:
np.broadcast_to(np.expand_dims(c,0), (3,3))

array([[10, 20, 30],
       [10, 20, 30],
       [10, 20, 30]])

In [29]:
c.shape

(3,)

In [30]:
np.expand_dims(c,0).shape

(1, 3)

The numpy `expand_dims` method lets us convert the 1-dimensional array `c` into a 2-dimensional array (although one of those dimensions has value 1).

In [31]:
np.expand_dims(c,0).shape

(1, 3)

In [32]:
m + np.expand_dims(c,0)

array([[11, 22, 33],
       [14, 25, 36],
       [17, 28, 39]])

In [33]:
np.expand_dims(c,1)

array([[10],
       [20],
       [30]])

`None` adds a new axis of length 1.

In [34]:
c[:, None].shape

(3, 1)

In [35]:
m + np.expand_dims(c,1)

array([[11, 12, 13],
       [24, 25, 26],
       [37, 38, 39]])

In [36]:
np.broadcast_to(np.expand_dims(c,1), (3,3))

array([[10, 10, 10],
       [20, 20, 20],
       [30, 30, 30]])

#### Broadcasting Rules

In [37]:
c[None]

array([[10, 20, 30]])

In [38]:
c[:,None]

array([[10],
       [20],
       [30]])

In [52]:
c[None] * c[:,None]    # outer product

array([[100, 200, 300],
       [200, 400, 600],
       [300, 600, 900]])

In [39]:
c[None] > c[:,None] # outer >

array([[False,  True,  True],
       [False, False,  True],
       [False, False, False]], dtype=bool)

In [40]:
xg,yg = np.ogrid[0:5, 0:5]; xg,yg

(array([[0],
        [1],
        [2],
        [3],
        [4]]), array([[0, 1, 2, 3, 4]]))

In [41]:
xg+yg

array([[0, 1, 2, 3, 4],
       [1, 2, 3, 4, 5],
       [2, 3, 4, 5, 6],
       [3, 4, 5, 6, 7],
       [4, 5, 6, 7, 8]])

**The RULES** 
When operating on two arrays, Numpy/PyTorch compares their shapes element-wise. 
It starts with the **trailing dimensions**, and works its way forward. 
Two dimensions are **compatible** when
- they are equal, or
- one of them is 1

Arrays do not need to have the same number of dimensions. 
For example, if you have a `256*256*3` array of RGB values, and you want to scale each color in the image by a different value, you can multiply the image by a one-dimensional array with 3 values. 
Lining up the sizes of the trailing axes of these arrays according to the broadcast rules, shows that they are compatible:

    Image  (3d array): 256 x 256 x 3
    Scale  (1d array):             3
    Result (3d array): 256 x 256 x 3

The [numpy documentation](https://docs.scipy.org/doc/numpy-1.13.0/user/basics.broadcasting.html#general-broadcasting-rules) includes several examples of what dimensions can and can not be broadcast together.

### Matrix Multiplication

We use broadcasting to define matrix multiplication.

In [53]:
m, c

(array([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]]), array([10, 20, 30]))

In [54]:
m @ c  # np.matmul(m, c)

array([140, 320, 500])

We get the same answer using `torch.matmul`:

In [55]:
torch.matmul(torch.from_numpy(m), torch.from_numpy(c))

tensor([ 140,  320,  500])

In [56]:
torch.from_numpy(m) @ torch.from_numpy(c)

tensor([ 140,  320,  500])

The following ` m * c` is **NOT** matrix multiplication.  What is it?

In [64]:
m,c 

(array([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]]), array([10, 20, 30]))

In [59]:
m * c   # element-wise with broadcasting

array([[ 10,  40,  90],
       [ 40, 100, 180],
       [ 70, 160, 270]])

In [60]:
(m * c).sum(axis=1)   # we sum this over the columns (axis=1) we get matrix vector product 

array([140, 320, 500])

In [48]:
c

array([10, 20, 30])

In [49]:
np.broadcast_to(c, (3,3))

array([[10, 20, 30],
       [10, 20, 30],
       [10, 20, 30]])

From a machine learning perspective, matrix multiplication is a way of creating features by saying how much we want to weight each input column.  **Different features are different weighted averages of the input columns**. 

The website [matrixmultiplication.xyz](http://matrixmultiplication.xyz/) provides a nice visualization of matrix multiplcation

In [50]:
n = np.array([[10,40],[20,0],[30,-5]]); n

array([[10, 40],
       [20,  0],
       [30, -5]])

In [51]:
m

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [52]:
m @ n

array([[140,  25],
       [320, 130],
       [500, 235]])

In [53]:
(m * n[:,0]).sum(axis=1)   

array([140, 320, 500])

In [54]:
(m * n[:,1]).sum(axis=1)

array([ 25, 130, 235])